# Scraping Google Reviews
The following notebook details how Google Maps reviews of parks in Montréal are being collected.

It is broken down into the following sections: 
<br>1. [Loading necessary libraries](#loading-lib)
<br>2. [Collecting park information](#collecting-park-info)
<br>3. [Collecting Google Maps data](#google-maps-calls)

<a id="loading-lib"></a>
## 1. Loading necessary libraries
One of the essential libraries for scraping web data is Selenium. To use Selenium, it first has to be installed as well as making sure that certain options are set for later scraping.

In [13]:
import pandas as pd # for later data processing

In [2492]:
# install necessary libraries 
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [2]:
# making sure that we are in the correct path 
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [1]:
# load selenium module for scraping 
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [6]:
# update options for scraping 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [1117]:
# create a webdriver browser instance to make website calls
#browser = webdriver.Chrome('chromedriver', options=chrome_options)

# if developing locally and not in Google Collab
browser = webdriver.Chrome(executable_path="/Users/andreamock/Documents/chromedriver")

<a id="collecting-park-info"></a>
## 2. Collecting park information 

### 2.1 Collecting names of parks 
There are multiple parks in Montréal. There are multiple approaches that can be taken in order to get a complete list of parks. The official website of Montréal contains a [list of parks](https://montreal.ca/lieux?mtl_content.lieux.category.code=PARC) in the city which will be used as the basis for collecting reviews for different parks. 

In [ ]:
# get the website with all the parks in Montréal
browser.get('https://montreal.ca/lieux?mtl_content.lieux.installation.code=PARC')

In [3]:
def gatherParkNames(browserEl):
    ''' Searches for the park names as well as urls to their corresponding site.
    Returns a list of tuples, where each tuple contains the park name and corresponding url'''

    # find_elements_by_xpath returns an array of selenium objects.
    park_elements = browserEl.find_elements_by_xpath('//div[@class="list-group list-group-teaser hub-list-group "]/a')

    # extract the links and names of the parks 
    all_park_info = []
    for p in park_elements: 
        park_name= p.text.split('\n')[0] # just extract park name, not part of city and address
        park_url = p.get_attribute('href')
        all_park_info.append((park_name, park_url))

    return all_park_info

In [4]:
def getAllParks(browserEl, numPages): 
    '''given the number of pages to traverse extracts all of the parks on the Montréal website and 
    returns the park name, the url to an information page for that park in the form of a list'''
    
    allparksInfo = [] # list to collect the information about all parks
    for i in range(numPages): 
        browserEl.get('https://montreal.ca/lieux?mtl_content.lieux.installation.code=PARC&page='+str(i+1))
        parksInfo = gatherParkNames(browserEl) # gather park information from one of the park overview pages
        allparksInfo = allparksInfo + parksInfo

    return allparksInfo

In [5]:
# gather all of the park names from the Montréal website
allParks = getAllParks(browser, 10)

In [6]:
# number of total parks 
len(allParks)

948

In [7]:
# sample entry of a park
allParks[0]

('Aire de repos 8e Avenue',
 'https://montreal.ca/lieux/aire-de-repos-8e-avenue')

### 2.2 Extracting additional park information 
The Montréal park overview site also has a designated page for each park which offers information about each park opening times, general information as well as a link to Google maps for the park. Therefore since we collected the name of each park as well as to the individual site designated to each park we use it as a way to gather additional information about the park for later if needed. 

In [3]:
def extractAdditionalParkInfo(parkInfo, browserEl):
    ''' Given a list of park names and corresponding urls, extracts the park's google maps url as well as description
    if present. Returns a list of tuples that contains the park name, the link to the park's site on the montreal  
    website and the description of the park 
    '''
    fullParkInfo = []
    for parkName, parkLink in parkInfo:
        browserEl.get(parkLink)
        try: 
            parkUrl = browserEl.find_elements_by_xpath(
                '//div[@class="list-item-content"]/div[@class="list-item-action mt-1"]/a')
            googleMapsUrl = parkUrl[0].get_attribute('href') #retrieve the url to Google Maps
        except: 
            googleMapsUrl = None
        
        # try extracting a description, if not present just set to None
        try: 
            # get a description of location
            description = browser.find_elements_by_xpath('//div[@class="content-module-stacked"]/div/p')[0].text
        
        except: 
            description = None
        
        fullParkInfo.append((parkName, parkLink, googleMapsUrl, description))
    return fullParkInfo

In [23]:
# extract additional park info
allParksData = extractAdditionalParkInfo(allParks, browser)

In [24]:
# sample collected data entry 
allParksData[:1]

[('Aire de repos 8e Avenue',
  'https://montreal.ca/lieux/aire-de-repos-8e-avenue',
  'https://www.google.com/maps/search/?api=1&query=Boulevard%20Saint-Joseph%20Lachine%20H8S%202M2%20Qu%C3%A9bec,%20Canada',
  'L’aire de repos de la 8e Avenue offre un point de vue sur le canal de Lachine.')]

Having collected data for all the parks in Montréal, before moving on to Google Maps it is important to saveguard the data. To do so, the collected data set will be saved in a CVS file and can be easily loaded in the next time without having to repeat the data collecting step again. 

In [28]:
# save park information in dataframe
park_df = pd.DataFrame(allParksData, columns=['name', 'link', 'google_maps', 'description'])
park_df.head()

,name,link,google_maps,description
0,Aire de repos 8e Avenue,https://montreal.ca/lieux/aire-de-repos-8e-avenue,https://www.google.com/maps/search/?api=1&quer...,L’aire de repos de la 8e Avenue offre un point...
1,Bassin de la Brunante,https://montreal.ca/lieux/bassin-de-la-brunante,https://www.google.com/maps/search/?api=1&quer...,Le bassin de la Brunante est un lieu privilégi...
2,Belvédère du Chemin-Qui-Marche,https://montreal.ca/lieux/belvedere-du-chemin-...,https://www.google.com/maps/search/?api=1&quer...,C’est un parc linéaire proche du fleuve Saint-...
3,Boisé du parc Marcel-Laurin,https://montreal.ca/lieux/boise-du-parc-marcel...,https://www.google.com/maps/search/?api=1&quer...,Consultez la carte des sentiers.
4,Boisé Saint-Conrad,https://montreal.ca/lieux/boise-saint-conrad,https://www.google.com/maps/search/?api=1&quer...,Venez profiter des attraits de la nature ou pr...


In [29]:
# save data in csv file
#park_df.to_csv('ParkInformation.csv')

<a id="google-maps-calls"></a>
## 3. Making Google API calls
After having collected the name of all the parks in Montréal, the next step is to search them up on Google Maps and extract the reviews for each park. 

For the collection of reviews Selenium will be utilized. In order to search for a particular park, one must first search up the name of the park, click on the reviews, scroll down to gather all reviews since the site is dynamically loaded and finally collect the reviews and store them. For each park the reviews will be stored in a csv file. 

In [19]:
# load in dataframe
park_df = pd.read_csv('ParkInformation.csv', index_col=0)
park_df.tail()

,name,link,google_maps,description
943,Square Saint-Patrick,https://montreal.ca/lieux/square-saint-patrick,NaN,Consultez cet article pour connaître l’horaire...
944,Square Sainte-Élisabeth,https://montreal.ca/lieux/square-sainte-elisabeth,https://www.google.com/maps/search/?api=1&quer...,Consultez cet article pour connaître l’horaire...
945,Square Sir-George-Étienne-Cartier,https://montreal.ca/lieux/square-sir-george-et...,https://www.google.com/maps/search/?api=1&quer...,Consultez cet article pour connaître l’horaire...
946,Square Victoria,https://montreal.ca/lieux/square-victoria,https://www.google.com/maps/search/?api=1&quer...,C’est un parc historique divisé en deux partie...
947,Square Viger,https://montreal.ca/lieux/square-viger,https://www.google.com/maps/search/?api=1&quer...,Venez en profiter!


In [942]:
# load necessary libraries
import time
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [5]:
def searchplace(browserEl, search):
    ''' finds the search bar and performes a search for a given search phrase in google maps'''
    place = browserEl.find_element_by_class_name("tactile-searchbox-input")
    place.clear()
    place.send_keys(search)
    submitButton = browserEl.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
    submitButton.click()

In [1091]:
def clickOnCorrectPlace(browserEl): 
    '''If a search result in Google Maps yields multiple results clicks on the first matching results to gather 
    reviews'''
    time.sleep(5)
    try: 
        parkInfo = browserEl.find_element_by_class_name("a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd")
        parkInfo.click()
    except: 
        print('Already correct link')

In [1106]:
def extractReviewNumbers(browserEl): 
    '''Given a google maps page extracts the number of reviews for particular place and returns it. If 
    the number of reviews is not stated returns 0 '''
    time.sleep(5)
    try:
        numReviews = browserEl.find_elements_by_xpath('//button[@class="widget-pane-link"]')[0].text
        reviewNums = int(re.findall('\d+',numReviews)[0])
    except:
        #print('Number of reviews not given')
        reviewNums = 0
    return reviewNums
            

In [1000]:
def goToAllReviews(browserEl):
    '''helper function that clicks on more reviews once a google maps place is loaded'''
    wait=WebDriverWait(browserEl, 10)
    try:
    #element = browserEl.find_elements_by_xpath('//button[@jsaction=\'pane.reviewlist.goToReviews\']')
        
        wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@jsaction=\'pane.reviewlist.goToReviews\']'))).click()
    #element[0].click()
    except:
        wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@jsaction=\'pane.reviewlist.goToReviews\']'))).click()
        

In [7]:
def scroll(browserEl): 
    '''scrolls down to dynamically load all of the reviews'''
    
    keepScrolling = True

    while keepScrolling: 
        time.sleep(2) # to allow for everything to load
        try: 
            # scroll down 
            scrollable_div = browserEl.find_element_by_css_selector('div.wo1ice-loading.noprint')
            browserEl.execute_script("arguments[0].scrollIntoView(true);", scrollable_div)
        except: 
            # once scrolled to the bottem print notification
            print('reached the end')
            keepScrolling = False

In [8]:
def expandReview(browserEl):
    '''Helper function that clicks open all reviews that are longer and for which the text is otherwise not 
    fully visible'''
    
    expandReviews = browserEl.find_elements_by_xpath('//button[@jsaction=\'pane.review.expandReview\']')

    for ex in expandReviews: 
        time.sleep(2)
        try: 
            ex.click()
        except:
            print('error expanding review pane')
    #print('All reviews expanded successfully')

In [9]:
def collectReviewInfo(review,reviewFor): 
    ''' 
    Given the html for a review, as well as the name of the park for which the review is (reviewFor) extracts the 
    review id, username, url of the contributer profile, published date, number of previous reviews user has, 
    number of stars and the text of the review
    if there and returns a dictionary with the review information 
    '''
    
    reviewInfo = {} # dictionary to store review information 
    
    review_id = review['data-review-id']
    username = review.find('div', class_='ODSEW-ShBeI-title').find('span').text
    user_url = review.find('a')['href']
    date_published = review.find('span', class_='ODSEW-ShBeI-RgZmSc-date').text
    num_stars = float(review.find('span', class_='ODSEW-ShBeI-H1e3jb')['aria-label'].split(' ')[1])
    
    try: # collect number of previous reviews by user if present
        review_nums = review.find('div', class_='ODSEW-ShBeI-VdSJob').find_all('span')[1].text
        num_reviews = int(re.findall('\d+', review_nums.split()[0])[0]) 
    except:
        num_reviews = 0
    
    try: # extract review text if present
        review_text = review.find('span', class_='ODSEW-ShBeI-text').text
    except Exception as e:
        review_text = None
    
    reviewInfo['review_for'] = reviewFor
    reviewInfo['review_id'] = review_id
    reviewInfo['username'] = username
    reviewInfo['user_url'] = user_url
    reviewInfo['published'] = date_published
    reviewInfo['date_retrieved'] = datetime.now()
    reviewInfo['num_stars'] = num_stars
    reviewInfo['num_reviews'] = num_reviews
    reviewInfo['review_text'] = review_text
    
    return reviewInfo

In [1107]:
def collectParkReviews(query, browserEl):
    '''Given the name of a park does a google maps search for the park, expands all reviews and scrapes them.
    If no park reviews are found None is returned, otherwise all of the reviews are returned in the form of a list'''
    
    
    time.sleep(2)
    searchplace(browserEl, query + ' Montréal') # searches for the park in search bar
    
    time.sleep(5)
    clickOnCorrectPlace(browserEl)
    
    try: 
        time.sleep(5) # leave time to load page
        
        numReviews = extractReviewNumbers(browserEl) # gets the number of reviews for a particular location
        print('Num reviews', numReviews)
        if numReviews == 0:
            my_file = open("noParkReviews.txt","a+") # adds te name of a park with no reviews to a txt file 
            my_file.write(query+ '\n')
            return None # no reviews found
        
        if numReviews > 3: 
            time.sleep(5)
            goToAllReviews(browserEl) # tries going to the review page
            time.sleep(2) 
            scroll(browserEl) # scrolls down
            time.sleep(2) # leave time to load page
        
        expandReview(browserEl) # expands long reviews
        time.sleep(3) # leave time to load page
        
        # use BeautifulSoup to parse and extract the information for each review 
        response = BeautifulSoup(browserEl.page_source, 'html.parser')
        rblock = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
        print('reached this point')
        allReviewData = [collectReviewInfo(ireview, query) for ireview in rblock]
        return allReviewData # return the list of collected review information 
    
    except:
        print('unable to collect reviews')
        return None

In [985]:
def collectMultipleReviews(parkList, browserEl):
    '''Given a list of multiple park names searches for reviews for all of the parks and for each successful 
    collection of a parks saves the reviews for that particular park in a csv file with the parks name. 
    If a park did not have any reviews or review collection was unsuccessful the names of these parks will be returned 
    as a list for further troubleshooting. 
    '''
    
    unsuccessful = []
    for park in parkList: 
        browserEl.get('https://www.google.com/maps')
        time.sleep(5)
        reviewsData = collectParkReviews(park, browserEl)
        if reviewsData is None: 
            unsuccessful.append(park)
        else:
            df = pd.DataFrame(reviewsData)
            df.to_csv(park + '.csv')
            
    return unsuccessful

In [931]:
browser.get('https://www.google.com/maps')

In [20]:
# list of some of the parks 
list(park_df['name'][930:950])

['Place Vauquelin',
 'Place Victor-Morin',
 'Promenade du Rail',
 'Promenade Père-Marquette',
 'Square Cabot',
 'Square Dalhousie',
 'Square des Frères-Charon',
 'Square Dézéry',
 'Square Dorchester',
 'Square du Nordet',
 'Square du Petit-Prince',
 'Square Phillips',
 'Square Saint-Louis',
 'Square Saint-Patrick',
 'Square Sainte-Élisabeth',
 'Square Sir-George-Étienne-Cartier',
 'Square Victoria',
 'Square Viger']

In [2493]:
# collect the reviews for a subset of parks 
collectMultipleReviews(ark_df['name'][:100], browser)

While scraping Google map reviews I realized that for some parks, especially those with over a thousand reviews Google fails to dynamically load all of the reviews. Therefore the alternative is to collect solely the reviews that are able to load. This involves slightly more manual labor and can be performed with the code below. Essential it is the same as above with the caveat that the scrolling only happens as long as it is possible. 

In [2478]:
''' # for scraping parks that fail to dynamically load with functions from above 
query = 'Parc Ménard'
search = query + ' Montréal'
browser.get('https://www.google.com/maps')
searchplace(browser, search)
goToAllReviews(browser)

i = 0
while i in range(1000): 
    scrollable_div = browser.find_element_by_css_selector('div.wo1ice-loading.noprint')
    browser.execute_script("arguments[0].scrollIntoView(true);", scrollable_div)
    i +=1

expandReview(browser)
resp = BeautifulSoup(browser.page_source, 'html.parser')
rblock1 = resp.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
allReviewData = [collectReviewInfo(ireview, query) for ireview in rblock1]
len(allReviewData)

df = pd.DataFrame(allReviewData)
df.to_csv(query + '.csv')
'''

<a id="google-maps-calls"></a>
## 4. Merging collected reviews
All of the Google Map reviews for each individual park is stored in a different csv file. For later analysis it will be most useful to have a large dataset containing all of the park data. Therefore we take all of the csv files and merge them to one large csv file. 

In [849]:
# importing libraries
import glob
import os

In [2372]:
# directory with all of the scraped park reviews 
dirName = '/Users/andreamock/Documents/scrapedParkReviews'

In [2494]:
# merging the files
joined_files = os.path.join(dirName, "*.csv")
  
# A list of all joined files 
joined_list = glob.glob(joined_files)

In [2486]:
# join files in a pandas dataframe
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df

,Unnamed: 0,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text
0,0,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUNpeGF6TTNnRRAB,Claudia,https://www.google.com/maps/contrib/1001449741...,7 months ago,2021-06-20 22:04:09.211296,4.0,107.0,One of the nicest entry points to this invitin...
1,1,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSURDOGEyMGpnRRAB,Nate Neel,https://www.google.com/maps/contrib/1121030547...,8 months ago,2021-06-20 22:04:09.212245,5.0,121.0,"Waterfront to fish or just relax, great place ..."
2,2,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUM4Nk9Ya3lnRRAB,Yucel Salimoglu,https://www.google.com/maps/contrib/1034180738...,11 months ago,2021-06-20 22:04:09.213178,4.0,79.0,Everything except the parking is good here.
3,3,Parc de la Capture-d'Ethan-Allen,ChZDSUhNMG9nS0VJQ0FnSUNVdWNUbE9REAE,COCO BEADZ,https://www.google.com/maps/contrib/1036060504...,a year ago,2021-06-20 22:04:09.214115,4.0,128.0,"Defenely the best park in Montreal East, Tetre..."
4,4,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUMwdHJDTm1nRRAB,Anna Maria Fiore,https://www.google.com/maps/contrib/1016779009...,a year ago,2021-06-20 22:04:09.215069,5.0,39.0,It's so peaceful and happy place near the water
...,...,...,...,...,...,...,...,...,...,...
88621,0,Parc du Belvédère,ChZDSUhNMG9nS0VJQ0FnSUNxOHBUbkRBEAE,Greg Domagala,https://www.google.com/maps/contrib/1014576480...,3 weeks ago,2021-06-24 21:54:50.493869,4.0,241.0,Close to home. Nice modules for younger kids. ...
88622,1,Parc du Belvédère,ChZDSUhNMG9nS0VJQ0FnSUNLMDdhU1dnEAE,Desmond Yu,https://www.google.com/maps/contrib/1174595133...,2 months ago,2021-06-24 21:54:50.494980,3.0,118.0,"Half basketball court on one side, and childre..."
88623,2,Parc du Belvédère,ChdDSUhNMG9nS0VJQ0FnSURjOFlDZzlBRRAB,Armen Kavaldjian,https://www.google.com/maps/contrib/1159419993...,a year ago,2021-06-24 21:54:50.496508,4.0,15.0,NaN
88624,3,Parc du Belvédère,ChdDSUhNMG9nS0VJQ0FnSURDOExfa3h3RRAB,Gabriel Zaurrini,https://www.google.com/maps/contrib/1089588150...,9 months ago,2021-06-24 21:54:50.498836,4.0,5.0,NaN


In [2490]:
merged_df = df.drop('Unnamed: 0', axis=1) # get rid of unnecessary column
merged_df.head()

,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text
0,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUNpeGF6TTNnRRAB,Claudia,https://www.google.com/maps/contrib/1001449741...,7 months ago,2021-06-20 22:04:09.211296,4.0,107.0,One of the nicest entry points to this invitin...
1,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSURDOGEyMGpnRRAB,Nate Neel,https://www.google.com/maps/contrib/1121030547...,8 months ago,2021-06-20 22:04:09.212245,5.0,121.0,"Waterfront to fish or just relax, great place ..."
2,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUM4Nk9Ya3lnRRAB,Yucel Salimoglu,https://www.google.com/maps/contrib/1034180738...,11 months ago,2021-06-20 22:04:09.213178,4.0,79.0,Everything except the parking is good here.
3,Parc de la Capture-d'Ethan-Allen,ChZDSUhNMG9nS0VJQ0FnSUNVdWNUbE9REAE,COCO BEADZ,https://www.google.com/maps/contrib/1036060504...,a year ago,2021-06-20 22:04:09.214115,4.0,128.0,"Defenely the best park in Montreal East, Tetre..."
4,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUMwdHJDTm1nRRAB,Anna Maria Fiore,https://www.google.com/maps/contrib/1016779009...,a year ago,2021-06-20 22:04:09.215069,5.0,39.0,It's so peaceful and happy place near the water


In [2491]:
merged_df.to_csv('ParkReviews.csv') # save merged dataframe to a csv file 

Finally we can also cross check and make sure that we scraped all parks. To do so, we first look at all of the parks that were successfully scraped and compare them with the list of parks for which no reviews were found.  

In [2443]:
# gather the list of parks who successfully were scraped
listOfFiles = os.listdir(dirName)
fileNames = [file.split('.csv')[0] for file in listOfFiles]

In [2444]:
# check if there are any parks that were unable to be scraped 
needScraping = [] # list of parks that were unable to be scraped 
for parkN in list(park_df['name']):
    if parkN not in fileNames and '/' not in parkN: 
        needScraping.append(parkN)

In [2447]:
needScraping[:10] # examople of parks for which no reviews were present/scraping was unsuccessful

['Aire de repos 8e Avenue',
 'Belvédère du Chemin-Qui-Marche',
 'Boisé Saint-Conrad',
 "Carré d'Hibernia",
 'Espace Faubourg Québec',
 'Espace Parthenais-Larivière',
 'Esplanade Tranquille',
 'Jardin Marcelle-Gauvreau',
 'Le Bouquiniste',
 'Le Champ-de-Mars']

In [2448]:
# read all of the parks for which no reviews were found
fileNoPark = open("noParkReviews.txt", 'r')
noParkInfo = fileNoPark.readlines()
noParkInfo1 = [parkinf.strip('\n') for parkinf in noParkInfo] 

In [2452]:
def diff(li1, li2):
    '''find the difference between two lists'''
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

In [2453]:
diff(noParkInfo1,needScraping) # check which parks still need to be scraped (ie which have no reviews)

['Parc Maurice-Cullen',
 'Parc Ludmilla-Chiriaeff',
 'Parc Mignault',
 'Parc des Madelinots',
 'Parc Mahatma-Gandhi',
 'Parc Maurice-Bélanger',
 'Parc Mia-Riddez-Morisset',
 'Parc Magnan',
 'Parc Marin',
 'Parc Lucien-Caron',
 'Parc Luc-Durand',
 'Parc Marguerite-Bourgeoys',
 'Parc Louise-Deschênes',
 'Parc Marian-Dale-Scott',
 'Parc Lucia-Kowaluk',
 'Parc des ÉpinettesParc des Hommes-Forts',
 'Jardin Marcelle-Gauvreau',
 'Parc Luigi-Pirandello',
 'Parc Marcel-Léger',
 'Parc Maria-Goretti',
 'Parc Ménard',
 'Parc Michel-Ménard',
 'Parc Martineau']